In [1]:
import numpy as np
from keras import Sequential
from keras.layers import GRU, Bidirectional, Embedding, Dense, Dropout, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
data = np.load('rnn-challenge-data.npz')
data_x = data['data_x']
data_y = data['data_y']

val_x = data['val_x']
val_x = val_x
val_y = data['val_y']

test_x = data['test_x']

In [3]:
def vectorise_sequence(sequence):
    alphabet = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    encoded = []
    for i, row in enumerate(sequence):
        seq_encoding = np.array([alphabet[ch] for ch in row])
        encoded.append(seq_encoding)
    return np.array(encoded)

In [4]:
from keras.callbacks import Callback

class TerminateOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='acc', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [5]:
dropout = 0.5
callback = EarlyStopping(monitor='val_categorical_accuracy', patience=0, baseline=1.0)


model = Sequential()
model.add(Embedding(input_dim=4, output_dim=64, name='embedding_layer'))
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(Dropout(dropout))
model.add(Bidirectional(GRU(128)))
model.add(Dropout(dropout))
model.add(Dense(5, activation='softmax'))
model.compile('adam', 'categorical_crossentropy', metrics=['categorical_accuracy'])

class_weight = class_weight.compute_class_weight('balanced', np.unique(data_y), data_y)
data_x_encoded = vectorise_sequence(data_x)
val_x_encoded = vectorise_sequence(val_x)

W0623 11:24:07.952186 140062235854656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0623 11:24:07.968384 140062235854656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0623 11:24:07.971403 140062235854656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0623 11:24:08.316542 140062235854656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0623 11:24:08.322863 

In [6]:
history = model.fit(data_x_encoded, to_categorical(data_y), batch_size=64, class_weight=class_weight,
                    epochs=100, verbose=1, callbacks=[TerminateOnBaseline(monitor='val_categorical_accuracy', baseline=1.0)], 
                    validation_data=(val_x_encoded, to_categorical(val_y)))

W0623 11:24:20.558073 140062235854656 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 400 samples, validate on 100 samples
Epoch 1/100
400/400 [==============================] - 20s 50ms/step - loss: 1.6085 - categorical_accuracy: 0.2125 - val_loss: 1.6026 - val_categorical_accuracy: 0.3100
Epoch 2/100
400/400 [==============================] - 16s 41ms/step - loss: 1.5988 - categorical_accuracy: 0.2775 - val_loss: 1.5929 - val_categorical_accuracy: 0.3300
Epoch 3/100
400/400 [==============================] - 16s 41ms/step - loss: 1.5806 - categorical_accuracy: 0.3250 - val_loss: 1.5731 - val_categorical_accuracy: 0.3200
Epoch 4/100
400/400 [==============================] - 16s 41ms/step - loss: 1.5488 - categorical_accuracy: 0.3225 - val_loss: 1.5335 - val_categorical_accuracy: 0.4200
Epoch 5/100
400/400 [==============================] - 16s 41ms/step - loss: 1.4873 - categorical_accuracy: 0.3625 - val_loss: 1.4621 - val_categorical_accuracy: 0.3200
Epoch 6/100
400/400 [==============================] - 16s 41ms/step - loss: 1.3836 - categorical_accuracy: 0

Epoch 49/100
400/400 [==============================] - 16s 41ms/step - loss: 0.6464 - categorical_accuracy: 0.6800 - val_loss: 0.5899 - val_categorical_accuracy: 0.6600
Epoch 50/100
400/400 [==============================] - 16s 41ms/step - loss: 0.6069 - categorical_accuracy: 0.7025 - val_loss: 0.5832 - val_categorical_accuracy: 0.6800
Epoch 51/100
400/400 [==============================] - 16s 41ms/step - loss: 0.6072 - categorical_accuracy: 0.7075 - val_loss: 0.5718 - val_categorical_accuracy: 0.7100
Epoch 52/100
400/400 [==============================] - 16s 41ms/step - loss: 0.6140 - categorical_accuracy: 0.7150 - val_loss: 0.5622 - val_categorical_accuracy: 0.7200
Epoch 53/100
400/400 [==============================] - 16s 41ms/step - loss: 0.6209 - categorical_accuracy: 0.6850 - val_loss: 0.5579 - val_categorical_accuracy: 0.7300
Epoch 54/100
400/400 [==============================] - 16s 41ms/step - loss: 0.5781 - categorical_accuracy: 0.7200 - val_loss: 0.5497 - val_categoric

In [7]:
test_x_encoded = vectorise_sequence(test_x)

In [8]:
y = model.predict(test_x_encoded)

In [16]:
y = np.array([np.argmax(arr) for arr in y])

In [17]:
y.shape

(250,)

In [18]:
np.save('prediction.npy', y)